In [1]:
%load_ext jupyter_black

In [2]:
import json
from glob import glob
from pathlib import Path
from typing import NewType, Iterable, Callable

import nvector
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import nvector as nv

# ml
import tensorflow
from sklearn.manifold import LocallyLinearEmbedding

# plotting
import matplotlib.pyplot as plt

# number of seconds in 2 mins
TWO_MINS = 120.0
idx: slice = pd.IndexSlice

FeatureCollection = NewType("FeatureCollection", dict)

wgs84 = nv.FrameE(name="WGS84")

# all_files = sorted(glob("/workspaces/sppp/data/probsevere/2021/**/*.json"))

2022-07-21 15:20:23.248103: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-21 15:20:23.248139: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [8]:
from sklearn.manifold import LocallyLinearEmbedding
from shapely.geometry import Polygon

from sppp.transfer.const import GridSpace


grid = GridSpace()


def embed(df: pd.DataFrame, grid: GridSpace) -> pd.DataFrame:

    lle = LocallyLinearEmbedding(n_components=1, n_neighbors=10)
    fresh = df[["PS", "MOTION_EAST", "MOTION_SOUTH"]].copy()
    # cent = df["geometry"].centroid
    fresh["STAB"] = lle.fit_transform(df[["MUCAPE", "MLCAPE", "MLCIN"]])
    # fresh = fresh.astype(np.float32)

    fresh["X"] = np.argmin(abs(df.X.values[:, np.newaxis] - grid.x), axis=1)
    fresh["Y"] = np.argmin(abs(df.Y.values[:, np.newaxis] - grid.y), axis=1)

    return fresh


# fresh = midf.pipe(embed, grid).


def x_any_y(df: pd.DataFrame) -> pd.DataFrame:
    def generator():
        for arr in df["geometry"]:
            point = Polygon(arr.tolist()).centroid
            yield point.x, point.y

    df[["X", "Y"]] = tuple(generator())

    return df


def open_parquet(path: str) -> pd.DataFrame:
    df = pd.read_parquet(path)
    mask = df.columns[~df.columns.isin(("MAXRC_EMISS", "MAXRC_ICECF", "geometry"))]
    df[mask] = df[mask].astype(np.float32)

    return df


midf = (
    open_parquet("/workspaces/sppp/data/2021-10.parquet")
    .pipe(x_any_y)
    .drop(columns=["geometry"])
    .pipe(embed, grid)
)

midf

PS  MOTION_EAST  MOTION_SOUTH      STAB     X  \
validTime           ID                                                       
2021-10-11 07:36:53 91366  15.0        8.415         1.715 -0.000279  3184   
                    92106   0.0       -0.573        -2.552 -0.000113  3257   
                    92451  72.0       10.628        -1.365 -0.000419  3453   
                    92565   0.0        5.947       -10.048 -0.000600  4389   
                    92609   1.0        3.173        -7.111 -0.003493  4394   
...                         ...          ...           ...       ...   ...   
2021-10-11 00:48:54 90328   4.0       12.263        -9.197  0.000151  3543   
                    90329   6.0        7.236        -9.496 -0.000378  3308   
                    90330   9.0       14.995        -9.057  0.000121  3189   
                    90331   1.0       -2.758        -1.557  0.000155  5002   
                    90332   0.0       -4.353        -4.430  0.000097  5100   

                              Y  
validTime           ID           
2021-10-11 07:36:53 91366  2539  
                    92106  1680  
                    92451  2399  
                    92565   822  
                    92609   781  
...                         ...  
2021-10-11 00:48:54 90328  1689  
                    90329  1820  
                    90330  1981  
                    90331  2606  
                    90332  2894  

[52206 rows x 6 columns]

In [10]:
# machine learning support classes
import keras

# from tensorflow import keras
from keras.engine.sequential import Sequential

# keras:K


class State:
    __has_state: bool = False

    def __init__(self):
        self.__latests: pd.DataFrame = None
        self.__state: pd.DataFrame = None

    def __repr__(self) -> str:
        return self.__state.__repr__()

    @property
    def latests(self) -> pd.DataFrame:
        return self.__latests

    def set_storm(self, df: pd.DataFrame) -> None:

        self.__latests = df

        if not self.__has_state:
            self.__has_state = True
            self.__state = df
        else:
            old = self.__state
            self.__state = pd.concat([old, df])

    def iterstorms(self):
        df = self.__state.iloc[-2:].groupby("ID")
        for id, x in self.__state.iloc[-2:].groupby("ID"):
            yield x

    def has_hist(self) -> bool:
        return isinstance(self.__state, pd.DataFrame)

    @property
    def frame(self) -> pd.DataFrame:
        return self.__state


def build_model(frame_b: pd.DataFrame) -> Sequential:
    model = keras.models.Sequential(
        [
            keras.layers.Dense(
                22.5,
                activation="elu",
                input_shape=frame_b.shape,
            ),
            keras.layers.Dense(
                22.5,
                activation="sigmoid",
            ),
            keras.layers.Dense(3),
        ]
    )
    return model


def make_reward_map() -> np.ndarray:
    """creates 11x11 matrix"""
    a = list(np.linspace(0, 0.25, 5))
    a2 = np.array(a + [0.5] + list(reversed(a)))
    return a2 + a2[:, np.newaxis]

In [17]:
from pettingzoo.sisl import pursuit_v4
import tensorflow as tf
import gym

from sppp.transfer.funcs import mask_frames_by_id
from sppp.transfer.const import GridSpace
from pettingzoo.butterfly import pistonball_v6

env = pistonball_v6.env()
zeros = grid.zeros_grid()


class SPPPEnv(gym.Env):
    def __init__(self, state: "State") -> None:
        self.state = state

    def _compute_reward(self) -> int:
        return 1

    def step(self, action: np.ndarray):
        # action is produced by DQN, action is discrete
        # self.cache.move(action)
        # compute reward based on state(position) of the car
        # storm_state = self.car_agent.getCarState()
        # reward = self._compute_reward(storm_state)
        # # check if the episode is done
        # car_controls = self.car_agent.getCarControls()
        # done = self._isDone(storm_state, car_controls, reward)
        # # log info
        # info = {}
        # # observation is RGB image from car's camera
        # observation = self.car_agent.observe()
        observation = self.state.latests
        reward = self._compute_reward()
        done = False
        info = {}
        return observation, reward, done, info

    @property
    def observation_space(self) -> pd.DataFrame:
        return self.state.latests


def iterframe(df: pd.DataFrame) -> Iterable[tuple[pd.Timestamp, pd.DataFrame]]:
    yield from df.groupby("validTime")


n_inputs = 4
state = State()
env = SPPPEnv(state)
loss = keras.losses.binary_crossentropy
reward_map = make_reward_map()


n_outputs = 5
box_shape = 5


def reward_matrix(x: int, y: int, box_size: int = 5):
    gamefield = (
        zeros.loc[
            y - box_size : y + box_size,
            x - box_size : x + box_size,
        ].copy()
        + reward_map
    )
    return gamefield.values


transition_probs = [[0, 0, 1] * 20] * 101
possible_actions = [[0, 1, 2], [0, 2], [1]] * 101  # [list(range(-5, 5))] * 101


def swp_class_ploicy(
    state: int,  # probiblity of severe weather
    epsilon=0,
):
    if np.random.rand() < epsilon:
        return np.random.randint()
    # v = model.predict(state[np.newaxis])
    return np.random.choice(possible_actions[state])


class SevereWeatherClassifcation:
    def policy(self, series: pd.Series) -> np.ndarray:
        return 1


swc = SevereWeatherClassifcation()

if __name__ == "__main__":
    for vt, df in iterframe(midf.loc[idx[:, "89519"], :]):
        track_rewards = 0
        obs = env.reset()
        # evaluate prediction
        if state.has_hist():
            # there is existing storm information normalize frame_a and frame_b by the the in's in the index
            frame_a, frame_b = mask_frames_by_id(state.latests, df)
            # assert that the frames are of an equal shape
            assert frame_a.shape == frame_b.shape
            # bg = frame_b[["MOTION_EAST", "MOTION_SOUTH", "STAB"]].values
            # x = play_one_step(env, bg, model, loss)
            # obs, reward, done, info = env.step(frame_b[["MOTION_EAST", "MOTION_SOUTH"]])
            with tf.GradientTape() as tape:
                for x, s in frame_b.iterrows():
                    model = build_model(s)
                    reward = grid.reward_matrix(s.X, s.Y)
                    action = swc.policy(s)
        state.set_storm(df)

reward

ModuleNotFoundError: No module named 'pygame'